<a href="https://colab.research.google.com/github/Beh-noush/General-Python/blob/main/FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple Neural Network with Pytorch

In [1]:
import torch
import torch.nn as nn

A fully-connected 2-layer network with Relu non-linearity.

In [20]:
import torch.nn.functional as F

In [21]:
F.relu

<function torch.nn.functional.relu>

In [28]:
class FCNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, out_dim):
    super(FCNN, self).__init__()
    self.preactivation_hidden = nn.Linear(input_dim,hidden_dim)
   # self.activation_hidden = nn.ReLU(self.preactivation_hidden)
    self.preactivation_out = nn.Linear(hidden_dim, out_dim)
  #  self.activation_hidden = nn.ReLU(self.preactivation_out)

  def forward(self, x):
    """
    Input:
          1-d array
          x is the input of the Neural Net of dimension input_dim
    Returns:
          1-d array of dimension out_dim. The score function output.
    """
    preactive_1 = self.preactivation_hidden(x)
    active_1 = F.relu(preactive_1)
    pre_output = self.preactivation_out(active_1)
    output = F.relu(pre_output)
    return output

# Why is this code wrong? 
#Ok, By replacing nn.ReLU with F.relu it works.
#It remains to check that both network structures are correct.


In [83]:
class FCNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, out_dim):
    super(FCNN, self).__init__()
    self.preactivation_hidden = nn.Linear(input_dim,hidden_dim)
    self.activation_hidden = nn.ReLU()
    self.preactivation_out = nn.Linear(hidden_dim, out_dim)
    self.activation_out = nn.Softmax()

  def forward(self, x):
    """
    Input:
          1-d array
          x is the input of the Neural Net of dimension input_dim
    Returns:
          1-d array of dimension out_dim. The score function output.
    """
    preactive_1 = self.preactivation_hidden(x)
    active_1 = self.activation_hidden(preactive_1)
    pre_output = self.preactivation_out(active_1)
    output = self.activation_out(pre_output)
    return output



In [78]:
my_net = FCNN(10, 15,2)


In [45]:
input_vector = torch.Tensor(3,10).uniform_(-1,1)

In [46]:
my_net.forward(input_vector)

tensor([[0.0000, 0.4418],
        [0.0000, 0.2924],
        [0.1829, 0.2840]], grad_fn=<ReluBackward0>)

In [7]:
my_net.preactivation_hidden

Linear(in_features=10, out_features=15, bias=True)

We use this neural net on Iris dataset.

In [34]:
from sklearn import datasets

In [37]:
iris = datasets.load_iris()

In [38]:
type(iris)

sklearn.utils.Bunch

In [43]:
type(iris['data'])

numpy.ndarray

In [49]:
type(iris.data)

numpy.ndarray

In [58]:
iris_data_torch = torch.from_numpy(iris.data)

In [59]:
iris_data_torch.shape

torch.Size([150, 4])

This data is now ready to be feed into the neeural net.


In [84]:
my_net = FCNN(4, 10, 3) 

Remember it was necessary to put .float() after your dataset, otherwise an error occured complaining about the dtype double rather than float! It seems a new error!

In [119]:
my_net.forward(iris_data_torch.float())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.6224, 0.2720, 0.1057],
        [0.6127, 0.2736, 0.1137],
        [0.6059, 0.2786, 0.1155],
        [0.6055, 0.2770, 0.1175],
        [0.6199, 0.2738, 0.1063],
        [0.6472, 0.2541, 0.0987],
        [0.6098, 0.2741, 0.1162],
        [0.6203, 0.2718, 0.1079],
        [0.5958, 0.2807, 0.1235],
        [0.6112, 0.2777, 0.1111],
        [0.6347, 0.2664, 0.0989],
        [0.6157, 0.2735, 0.1108],
        [0.6056, 0.2803, 0.1141],
        [0.5840, 0.2921, 0.1238],
        [0.6435, 0.2650, 0.0915],
        [0.6559, 0.2533, 0.0907],
        [0.6404, 0.2596, 0.1000],
        [0.6263, 0.2668, 0.1068],
        [0.6518, 0.2542, 0.0940],
        [0.6300, 0.2662, 0.1038],
        [0.6362, 0.2629, 0.1009],
        [0.6333, 0.2609, 0.1058],
        [0.6002, 0.2856, 0.1142],
        [0.6378, 0.2522, 0.1101],
        [0.6209, 0.2693, 0.1098],
        [0.6194, 0.2693, 0.1113],
        [0.6298, 0.2602, 0.1099],
        [0.6272, 0.2690, 0.1037],
        [0.6249, 0.2701, 0.1050],
        [0.611

Now, we can train our neural net on this dataset.

In [88]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [90]:
list(my_net.parameters()) #It contains weights and biases.

[Parameter containing:
 tensor([[ 0.4943,  0.1262,  0.1936,  0.3627],
         [ 0.2767, -0.2441, -0.4087, -0.1398],
         [-0.2096,  0.4884,  0.2511, -0.1610],
         [ 0.4618, -0.1699, -0.1292, -0.1604],
         [ 0.1158,  0.1164,  0.0487, -0.3122],
         [-0.3074, -0.2158, -0.0487, -0.1790],
         [-0.1393,  0.4505,  0.0704, -0.4620],
         [ 0.0299,  0.1465,  0.1960, -0.0678],
         [-0.4093,  0.3419, -0.4740, -0.1335],
         [ 0.3097,  0.2232, -0.4883, -0.1055]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4656, -0.1151, -0.4064, -0.0455, -0.1885,  0.4079, -0.1187,  0.2300,
         -0.2814, -0.0677], requires_grad=True),
 Parameter containing:
 tensor([[ 0.0784,  0.0656,  0.1392, -0.1270,  0.1772, -0.1486, -0.2591,  0.2735,
          -0.2024,  0.1350],
         [-0.1486,  0.0379, -0.1168,  0.0588,  0.0108, -0.1088,  0.2974,  0.1194,
          -0.1321,  0.0282],
         [-0.0393, -0.1958,  0.0550, -0.1562, -0.2573,  0.3075, -0.0185, -0.1962,
     

In [141]:
loss(output, iris_labels_torch)

tensor(1.1720, grad_fn=<NllLossBackward>)

In [149]:
#First, we train with unnormalized data.
epochs = range(20)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(my_net.parameters(), lr=1e-1)
iris_train = iris_data_torch.float()
iris_labels_torch = torch.from_numpy(iris.target)
iris_train.requires_grad = True
for e in epochs:
   optimizer.zero_grad()
   output = my_net.forward(iris_train)
   predictions = torch.argmax(output, axis = 1)
   iris_loss = loss(output, iris_labels_torch)
   iris_loss.backward()
   optimizer.step()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [150]:
predictions

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
        2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [135]:
import torch.optim